# by *convention* documents should use real data.

__pidgin.template__ supports jinja2 syntax to data in code cells before they execute.

## Python Line Transformer

In [ ]:
    from IPython.core.error import InputRejected
    from traceback import format_exception_only
    from IPython import get_ipython
    from IPython.core.inputtransformer import InputTransformer
    from jinja2 import Environment, TemplateError
    from collections import UserList
    from dataclasses import dataclass, field

    @dataclass
    class Template(InputTransformer, UserList):
        environment = Environment()
        user_ns: dict = field(default=None, repr=False)
        data: list = field(default_factory=list)
        
        def __post_init__(self): 
            if self.user_ns is None: self.user_ns = get_ipython().user_ns
            
        def push(self, line): self.data.append(line)
        
        def reset(self, *, code=""""""):
            while self.data: 
                code += self.data.pop(0) + "\n"
            try:
                return self.environment.from_string(code).render(**self.user_ns)
            except TemplateError as Exception:
                # http://ipython.readthedocs.io/en/stable/config/inputtransforms.html#string-based-transformations
                raise SyntaxError(format_exception_only(type(Exception), Exception))
        
        def __enter__(self,): 
            get_ipython().input_transformer_manager.python_line_transforms.insert(0, self)
        
        def __exit__(self, *args): 
            get_ipython().input_transformer_manager.python_line_transforms = list(
                filter(self.__ne__, get_ipython().input_transformer_manager.python_line_transforms))

## Incremental Importing of Templated Documents

TemplateLoader has to execute its own module and cache it in exec module.  Compile does the caching.  Compile needs to be called after the ast is evaluated.

In [1]:
    from importnb.loader import Notebook, AST

In [3]:
    class Incremental(AST):
        def from_code_cell(Incremental, cell, **dict):
            module = dict.pop('module')
            Module = super(type(Incremental), Incremental).from_code_cell(cell, **dict)
            
            if Module:
                eval(Incremental.compile(Module), module.__dict__, module.__dict__)
            return Module
        
        def from_notebook_node(AST, nb, resource: dict=None, **dict):         
            ast = super().from_notebook_node(nb, resource, **dict)
            return AST.compile(ast) and ast 

In [4]:
    class ImportTemplate(Notebook):
        def exec_module(Loader, module):
            from IPython.utils.capture import capture_output    
            with capture_output(stdout=False, stderr=False) as output: 
                try: 
                    with Template(user_ns=module.__dict__):
                        parser = Incremental(Loader.path, Loader.name)
                        with __import__('io').BytesIO(Loader.get_data(Loader.path)) as data:
                            parser.compile(parser.from_file(data, module=module))
                            
                except type('pass', (BaseException,), {}): ...
                finally: module.__output__ = output
            return module

## IPython Extensions

    %unload_ext pidgin.template

In [5]:
    def unload_ipython_extension(ip=None):
        ip = ip or get_ipython()
        ip.input_transformer_manager.python_line_transforms = [
            object for object in ip.input_transformer_manager.python_line_transforms
            if not isinstance(object, Template)
        ]
        ImportTemplate().__exit__()
    

    %reload_ext pidgin.template

In [6]:
    def load_ipython_extension(ip=get_ipython()):
        ip.input_transformer_manager.python_line_transforms.insert(0, Template())
        ImportTemplate().__exit__()
        ImportTemplate().__enter__()
        

## Testing

In [7]:
    class Test(__import__('unittest').TestCase): 
        def setUp(Test):
            %reload_ext pidgin
            load_ipython_extension()
            from nbformat import write, v4
            with open('test_template.ipynb', 'w') as file:
                write(v4.new_notebook(cells=[
                    v4.new_code_cell("""if __name__ == '__main__':
                    %reload_ext template"""),
                    v4.new_code_cell("""a=42"""),
                    v4.new_code_cell("""b={{a}}*10""")
                ]), file)
                
        def runTest(Test):
            global test_template
            import test_template
            assert test_template.__file__.endswith('.ipynb')
            assert test_template.a is 42
            assert test_template.b == 420
            
        def tearDown(Test):
            %rm test_template.ipynb
            unload_ipython_extension()

## Developer

In [8]:
    if __name__ == '__main__': 
        reporter = __import__('unittest').TextTestRunner()
        reporter.run(Test())
        __import__('doctest').testmod(verbose=2, report=reporter)

.

18 items had no tests:
    __main__
    __main__.ImportTemplate
    __main__.ImportTemplate.exec_module
    __main__.Incremental
    __main__.Incremental.from_code_cell
    __main__.Incremental.from_notebook_node
    __main__.Template
    __main__.Template.__enter__
    __main__.Template.__exit__
    __main__.Template.__post_init__
    __main__.Template.push
    __main__.Template.reset
    __main__.Test
    __main__.Test.runTest
    __main__.Test.setUp
    __main__.Test.tearDown
    __main__.load_ipython_extension
    __main__.unload_ipython_extension
0 tests in 18 items.
0 passed and 0 failed.
Test passed.



----------------------------------------------------------------------
Ran 1 test in 0.149s

OK
